In [1]:
import pandas as pd
from weblogo import *

### Configuration

In [2]:
sea = "sea"  # change to specific directory of MEME/bin/sea
df_name = "variant_T.csv"  # input, be careful for multiindex
motif_col = "motif_F10"
cluster_col = "Cluster"
meme_fn = "variant.meme"
fasta_fn = "variant.fa"
sea_out = "sea_out"

### Run

In [3]:
df = pd.read_csv(df_name, index_col=None, header=[0])
# df = pd.read_csv(df_name, index_col=[0,1,2], header=[0, 1]) # for dual header

In [4]:
df

,Unnamed: 0,Tan et al,motif_F10,base,num_of_sample,known_mod,X,Y,Cluster
0,chr1@91853111@-,True,TGACTGGGGCTGTACACCTGT,T,631,NaN,17.113300,5.654562,2
1,chr5@170819971@+,True,ATGAAGAGGATGATGATGAAG,T,360,NaN,12.605336,8.053991,3
2,chr19@12831754@-,True,CTGCAGCAGGTCCTGCAGCTG,T,352,NaN,13.990366,1.694899,10
3,chr16@89628783@+,True,ACCGGACCGGTCATGCCCGTC,T,295,NaN,14.936040,2.291367,11
4,chr16@31201633@+,True,GTGGTGGCAGTGGTGGTGGTG,T,230,NaN,15.958023,2.580020,11
...,...,...,...,...,...,...,...,...,...
1907,chr4@41992717@+,True,TAGCTGGTCCTCCCTGTGCCG,T,6,NaN,13.228830,2.456326,5
1908,chr16@418338@-,True,CAGAGACTTTTGGAGGAGAAG,T,6,NaN,14.529934,4.920469,11
1909,chr10@32311779@-,True,TTCGTATCTCTCAAGTATGTT,T,6,NaN,12.359640,5.530034,4
1910,Hs_18S@1@+,True,TTAATTTGACTCAACACGGGA,T,-1,m1acp3Psi,16.706734,4.942567,2


In [5]:
def extact_all_fasta(df_in, column, fn_out, rna=True):
    N = 0
    with open(fn_out, "w") as output:
        for idx, row in df_in.iterrows():
            if rna == True:
                output.write(">{}\n{}\n".format(N, row[column].replace("T", "U")))
            else:
                output.write(">{}\n{}\n".format(N, row[column]))
            N += 1        

In [6]:
extact_all_fasta(df, motif_col, fasta_fn)

In [7]:
def generate_meme_file(df_in, id_column, motif_column, fn_out, rna=True):
    all_ids = set(df_in[id_column].tolist())
    temp_mat_name = fn_out+".temp.mat"
    print(all_ids)
    with open(temp_mat_name, "w") as output:
        for ID in all_ids:
            print(ID)
            subdf = df_in[df_in[id_column]==ID]
            count_data = {}
            for _, row in subdf.iterrows():
                if "N" in row[motif_column]:
                    continue
                if rna == True:
                    iterseq = row[motif_column].replace("T", "U")
                else:
                    iterseq = row[motif_column]
                for idx, base in enumerate(list(iterseq)):
                    if idx not in count_data:
                        count_data[idx] = {"A":0, "C": 0, "G": 0, "U":0}
                    count_data[idx][base] += 1
            count_df = pd.DataFrame.from_dict(count_data).T
            seqs = count_df.values
            seqs = np.array(seqs)
            logodata = LogoData.from_counts(counts=seqs, alphabet='ACGU')

            temp = []
            for i in range(logodata.counts.shape[0]):
                # temp.extend(list(logodata.entropy[i]*logodata.counts[i]/logodata.counts[i].sum()))
                output.write("{}\t{}\t{}\t{}\n".format(logodata.counts[i][0], logodata.counts[i][1], logodata.counts[i][2], logodata.counts[i][3]))
            output.write("\n")
        
    !matrix2meme -rna < $temp_mat_name > $fn_out

In [8]:
generate_meme_file(df, cluster_col, motif_col, meme_fn)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
1
2
3
4
5
6
7
8
9
10
11


In [9]:
!$sea -oc $sea_out -p $fasta_fn -m $meme_fn

# Checking alphabets in 1 motif files.
# Loading motifs from file 'variant.meme'
# Alphabet: RNA
# NOTE: Will convert any DNA sequences to RNA.
# Positive sequences "variant.fa" - training: 1721 hold-out: 191
# Negative sequences are shuffled primary sequences (2-order) - training: 1721 hold-out: 191
# Estimating background model from control sequences.
# Background: A 0.221 C 0.213 G 0.29 U 0.276
# Background order: 2 Background size: 84
# Using Fisher Exact test for p-values.
# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 11. Using pi_zero = 1.0.
# Freeing storage...


### It is better to read the html.

In [10]:
df_sea = pd.read_csv("./{}/sea.tsv".format(sea_out), header=0, sep="\t")

In [11]:
df_sea

,RANK,DB,ID,ALT_ID,CONSENSUS,TP,TP%,FP,FP%,ENR_RATIO,SCORE_THR,PVALUE,LOG_PVALUE,EVALUE,LOG_EVALUE,QVALUE,LOG_QVALUE
0,1,variant.meme,11.0,NNNNNNNNBSUGNNVNNNNNN,VNVNNVVNGGUGGGSVVSVVS,748.0,43.46,189.0,10.98,3.94,0.110,7.510000e-107,-244.36,8.260000e-106,-241.96,8.260000e-106,-241.96
1,2,variant.meme,6.0,NNNHNNNDDRUDWRDNNDDDN,DDHHNWDDDRUWWRWDAADWN,507.0,29.46,168.0,9.76,3.01,0.011,9.610000e-50,-112.87,1.060000e-48,-110.47,5.280000e-49,-111.16
2,3,variant.meme,9.0,NNNNNNHNDBUDKNNNNNNDN,NHNNNHHNKBUUUKNNDNNDW,539.0,31.32,200.0,11.62,2.69,0.110,2.510000e-46,-105.00,2.760000e-45,-102.60,9.200000e-46,-103.70
3,4,variant.meme,5.0,BNNNBBNNYCUCSYNVVNSVN,BBBBBBSVCCUCCCVGSBSCN,356.0,20.69,98.0,5.69,3.61,0.210,1.380000e-40,-91.78,1.520000e-39,-89.38,3.790000e-40,-90.77
4,5,variant.meme,8.0,NHHNHHNCHNUCHMMNVVNHN,NMHCYHYCHMUCCCMNVMHYM,289.0,16.79,91.0,5.29,3.15,0.080,3.290000e-28,-63.28,3.620000e-27,-60.88,7.250000e-28,-62.49
5,6,variant.meme,2.0,NDNNNBDDDGUGDKNBWKNND,KGUGUGUGKGUGUGUGUGUGU,332.0,19.29,128.0,7.44,2.58,0.280,2.800000e-25,-56.54,3.080000e-24,-54.14,5.130000e-25,-55.93
6,7,variant.meme,7.0,AAAAAAAAAGURDDDNDNNNN,AAAAAAAAAGURWDDDDDNHU,158.0,9.18,37.0,2.15,4.18,0.084,2.780000e-20,-45.03,3.060000e-19,-42.63,4.370000e-20,-44.58
7,8,variant.meme,3.0,VNDVWRWARRUVAARAWRAWR,AWAAWRAAAAURAAAAARAAA,180.0,10.46,51.0,2.96,3.48,0.460,1.740000e-19,-43.19,1.920000e-18,-40.80,2.390000e-19,-42.88
8,9,variant.meme,10.0,YNSCDGCNKCUGCHGCNGSNG,CDGCDGCNGCUGCHGCWGCHG,148.0,8.60,48.0,2.79,3.04,0.940,5.020000e-14,-30.62,5.520000e-13,-28.22,6.140000e-14,-30.42
9,10,variant.meme,4.0,NUUYUUBYKBUHNDDUDWNDD,YUUUUUUUUYUWDUUUWUWUD,199.0,11.56,100.0,5.81,1.98,0.400,1.120000e-09,-20.61,1.230000e-08,-18.22,1.230000e-09,-20.52
